In [2]:
#Importing library

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from scipy import stats
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_squared_error
from math import sqrt


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
#Importing dataset

df = pd.read_csv("dataset/train_processed.csv")

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

from sklearn.ensemble import ExtraTreesRegressor

df.head(5)
df_clear=df.drop(['Events','StoreType','AssortmentType','Date','Max_Gust_SpeedKm_h'], axis=1)
df_clear=df_clear.dropna(axis=0)
df_clear.shape

In [5]:
df.head(5)

,Unnamed: 0,StoreID,Date,IsHoliday,IsOpen,HasPromotions,StoreType,NearestCompetitor,NumberOfCustomers,NumberOfSales,...,Hail,Thunderstorm,IsSaturday,IsSunday,WasOpenYesterday,IsOpenTomorrow,YesterdaySales,Previous3DaysSales,PreviousWeekSales,PreviousMonthSales
0,29,1000,2016-03-31,0,1,1,2,326,846,11907,...,0,0,0,0,1,0,9055,8731.333333,6875.714286,6490.517241
1,30,1000,2016-04-01,1,0,1,2,326,0,0,...,0,0,0,0,1,1,11907,9850.333333,7428.000000,6705.379310
2,31,1000,2016-04-02,0,1,0,2,326,851,11129,...,0,0,1,0,0,0,0,6987.333333,6300.857143,6425.689655
3,32,1000,2016-04-03,0,0,0,2,326,0,0,...,0,0,0,1,1,0,11129,7678.666667,7032.857143,6582.466667
4,33,1000,2016-04-04,1,0,0,2,326,0,0,...,0,0,0,0,0,1,0,3709.666667,7032.857143,6305.800000


In [6]:
df.shape

(501300, 49)

In [7]:
df_date=df['Date']
df_no_date=df.drop(['Date'], axis=1)

In [8]:
df_norm= (df_no_date - df_no_date.min()) / (df_no_date.max() - df_no_date.min())

In [9]:
df=df_norm
df['Date']=df_date
df.head(5)

,Unnamed: 0,StoreID,IsHoliday,IsOpen,HasPromotions,StoreType,NearestCompetitor,NumberOfCustomers,NumberOfSales,Region_AreaKM2,...,Thunderstorm,IsSaturday,IsSunday,WasOpenYesterday,IsOpenTomorrow,YesterdaySales,Previous3DaysSales,PreviousWeekSales,PreviousMonthSales,Date
0,0.000000,0.0,0.0,1.0,1.0,0.666667,0.003281,0.383500,0.446943,0.291715,...,0.0,0.0,0.0,1.0,0.0,0.339890,0.363301,0.355788,0.372606,2016-03-31
1,0.000002,0.0,1.0,0.0,1.0,0.666667,0.003281,0.000000,0.000000,0.291715,...,0.0,0.0,0.0,1.0,1.0,0.446943,0.409861,0.384367,0.384941,2016-04-01
2,0.000004,0.0,0.0,1.0,0.0,0.666667,0.003281,0.385766,0.417740,0.291715,...,0.0,1.0,0.0,0.0,0.0,0.000000,0.290735,0.326042,0.368885,2016-04-02
3,0.000006,0.0,0.0,0.0,0.0,0.666667,0.003281,0.000000,0.000000,0.291715,...,0.0,0.0,1.0,1.0,0.0,0.417740,0.319501,0.363920,0.377885,2016-04-03
4,0.000008,0.0,1.0,0.0,0.0,0.666667,0.003281,0.000000,0.000000,0.291715,...,0.0,0.0,0.0,0.0,1.0,0.000000,0.154355,0.363920,0.362002,2016-04-04


In [10]:
df_val3=df.loc[(df['Date']>='2018-01-01')]
df_val3_rest=df.loc[(df['Date']<'2018-01-01')]
df_val2=df_val3_rest.loc[(df_val3_rest['Date']>='2017-11-01')]
df_val2_rest=df.loc[(df['Date']<'2017-11-01')]
df_val1=df_val2_rest.loc[(df_val2_rest['Date']>='2017-09-01')]
df_train=df.loc[(df['Date']<'2017-09-01')]

In [11]:
df_train_clear=df_train.drop(['Date'], axis=1)
df_val1_clear=df_val1.drop(['Date'], axis=1)

In [12]:
df_store=df_train_clear.loc[(df_train_clear['StoreID']==0.0)]
df_val1_store=df_val1_clear.loc[(df_val1_clear['StoreID']==0.0)]
y=df_store['NumberOfSales']
y_val1=df_val1_store['NumberOfSales']
X=df_store.drop(['StoreID','NumberOfSales','NumberOfCustomers','YesterdaySales','Previous3DaysSales','PreviousWeekSales','PreviousMonthSales'], axis=1)
X_val1=df_val1_store.drop(['StoreID','NumberOfSales','NumberOfCustomers','YesterdaySales','Previous3DaysSales','PreviousWeekSales','PreviousMonthSales'], axis=1)
df_norm.head(4)

,Unnamed: 0,StoreID,IsHoliday,IsOpen,HasPromotions,StoreType,NearestCompetitor,NumberOfCustomers,NumberOfSales,Region_AreaKM2,...,Thunderstorm,IsSaturday,IsSunday,WasOpenYesterday,IsOpenTomorrow,YesterdaySales,Previous3DaysSales,PreviousWeekSales,PreviousMonthSales,Date
0,0.000000,0.0,0.0,1.0,1.0,0.666667,0.003281,0.383500,0.446943,0.291715,...,0.0,0.0,0.0,1.0,0.0,0.339890,0.363301,0.355788,0.372606,2016-03-31
1,0.000002,0.0,1.0,0.0,1.0,0.666667,0.003281,0.000000,0.000000,0.291715,...,0.0,0.0,0.0,1.0,1.0,0.446943,0.409861,0.384367,0.384941,2016-04-01
2,0.000004,0.0,0.0,1.0,0.0,0.666667,0.003281,0.385766,0.417740,0.291715,...,0.0,1.0,0.0,0.0,0.0,0.000000,0.290735,0.326042,0.368885,2016-04-02
3,0.000006,0.0,0.0,0.0,0.0,0.666667,0.003281,0.000000,0.000000,0.291715,...,0.0,0.0,1.0,1.0,0.0,0.417740,0.319501,0.363920,0.377885,2016-04-03


In [13]:
df_store.shape

(519, 48)

In [22]:
columns=['num_of_trees','depth','num_of_attr','mse']
index=range(1,200000)
hp=pd.DataFrame(index=index,columns=columns)
hp.head(5)

,num_of_trees,depth,num_of_attr,mse
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN


In [23]:
count=0
for depth in range(1,100,2):
    for nt in range(1,250,2):
        for na in range (1,42,2):
            forest = RandomForestRegressor(max_depth=depth, random_state=0, n_estimators=nt, max_features=na)
            forest.fit(X, y)
            y_pred=forest.predict(X_val1)
            mse_val1=mean_squared_error(y_val1,y_pred)
            hp.at[count,'num_of_trees']=nt
            hp.at[count,'num_of_attr']=na
            hp.at[count,'depth']=depth
            hp.at[count,'mse']=sqrt(mse_val1)
            count+=1

In [24]:
##0,16sec a modello

In [25]:
count

131250

In [26]:
hp.to_csv('randomforest.csv')

In [27]:
hp['mse'].min()

0.03084975139994731

In [30]:
hp_under_31=hp.loc[(hp['mse']<0.031)]
print(hp_under_31.shape)
print(hp_under_31.head(3))

(3, 4)
      num_of_trees depth num_of_attr        mse
26337            9    21           7  0.0308498
26589           33    21           7  0.0309453
31839           33    25           7  0.0308803


In [36]:
forest = RandomForestRegressor(max_depth=50, random_state=0, n_estimators=100, max_features=15)
forest.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
           max_features=15, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

In [37]:
X.head(5)

,Unnamed: 0,IsHoliday,IsOpen,HasPromotions,StoreType,NearestCompetitor,Region_AreaKM2,Region_GDP,Region_PopulationK,CloudCover,...,AssortmentType_With Non-Food Department,Rain,Snow,Fog,Hail,Thunderstorm,IsSaturday,IsSunday,WasOpenYesterday,IsOpenTomorrow
0,0.000000,0.0,1.0,1.0,0.666667,0.003281,0.291715,0.515529,0.266576,0.583787,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.000002,1.0,0.0,1.0,0.666667,0.003281,0.291715,0.515529,0.266576,0.583787,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.000004,0.0,1.0,0.0,0.666667,0.003281,0.291715,0.515529,0.266576,0.583787,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.000006,0.0,0.0,0.0,0.666667,0.003281,0.291715,0.515529,0.266576,0.486489,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.000008,1.0,0.0,0.0,0.666667,0.003281,0.291715,0.515529,0.266576,0.194596,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
print(forest.feature_importances_)

In [ ]:
forest.score(X, y)

In [ ]:
missing_values = round(df.isnull().sum() / df.shape[0] * 100, 3)
features_with_missings = missing_values.drop(missing_values[missing_values == 0].index).sort_values(ascending=False)
features_with_missings

In [ ]:
y_pred=forest.predict(X_val1)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
mse_val1=mean_squared_error(y_val1,y_pred)
print (sqrt(mse_val1))